<a href="https://colab.research.google.com/github/arthurvale/TCC-2026/blob/main/An%C3%A1lise_Explorat%C3%B3ria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Análise do dataset.

In [3]:
df = pd.read_parquet('/content/drive/MyDrive/5g final/dados/dataframe_cache.parquet')
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8144644 entries, 0 to 8144643
Data columns (total 37 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   Date             object 
 2   Time             object 
 3   UTC              float64
 4   Latitude         float64
 5   Longitude        float64
 6   Altitude         float64
 7   Speed            float64
 8   EARFCN           int64  
 9   Frequency        float64
 10  Band             int64  
 11  PCI              int64  
 12  SSBIdx           int64  
 13  SSS-SINR         float64
 14  SSS-RSRP         float64
 15  SSS-RSRQ         float64
 16  SSS-RePower      float64
 17  MNC              object 
 18  DM_RS-SINR       float64
 19  DM_RS-RSRP       float64
 20  DM_RS-RSRQ       float64
 21  DM_RS-RePower    float64
 22  PBCH-SINR        float64
 23  PBCH-RSRP        float64
 24  PBCH-RSRQ        float64
 25  PBCH-RePower     float64
 26  PSS-SINR         float64
 27  PSS-RSRP    

None

Descrição das Variáveis Categóricas (Artigo Referência):

- 'MNC': Identificador da operadora de rede móvel
- 'scenario': Contexto da coleta de dados:
  * 'OW' (Outdoor Walking): Coleta realizada a pé em ambiente externo.
  * 'OD' (Outdoor Driving): Coleta em veículo em movimento (cenário dinâmico).
  * 'IS' (Indoor Static): Coleta estática em ambiente interno.
- 'campaign': Subcampanhas de medição realizadas em diferentes dias e condições.
  Cada subcampanha é identificada por um código único (ex: 'OD_1', 'OD_gaming').



In [4]:
unicos_mnc = df['MNC'].unique()
print(unicos_mnc)
print('------------------------------------------------------------------------')
unicos_scenario = df['scenario'].unique()
print(unicos_scenario)
print('------------------------------------------------------------------------')
unicos_campaign = df['campaign'].unique()
print(unicos_campaign)

['"Op"[2]' '"Op"[1]']
------------------------------------------------------------------------
['OW' 'IS' 'OD']
------------------------------------------------------------------------
['campaign_6_OW_5G_gaming' 'campaign_35_IS_5G_speedtest'
 'campaign_10_OD_5G_speedtest' 'campaign_28_OW_5G_gaming'
 'campaign_20_OW_5G_gaming' 'campaign_18_OD_5G_gaming'
 'campaign_7_OD_5G_gaming' 'campaign_5_OW_5G_gaming'
 'campaign_29_OD_5G_gaming' 'campaign_4_OD_5G_gaming'
 'campaign_23_OD_5G_gaming' 'campaign_10_IS_5G_gaming'
 'campaign_27_IS_5G_gaming' 'campaign_9_IS_5G_gaming'
 'campaign_8_IS_5G_gaming' 'campaign_13_OW_5G_gaming'
 'campaign_24_OW_5G_gaming' 'campaign_31_OW_5G_gaming'
 'campaign_21_IS_5G_gaming' 'campaign_22_IS_5G_gaming'
 'campaign_33_IS_5G_gaming' 'campaign_12_OD_5G_gaming'
 'campaign_26_OD_5G_gaming' 'campaign_2_OD_5G_gaming'
 'campaign_14_IS_5G_gaming' 'campaign_15_IS_5G_gaming'
 'campaign_1_IS_5G_gaming' 'campaign_11_IS_5G_gaming'
 'campaign_34_IS_5G_gaming' 'campaign_32_IS_5G_

Nosso estudo de caso é sobre a classificação de Beamforming, neste contexto o tipo de campanha que oferece mais diversidade de condições de rede é a campanha com o veículo em movimento, OD, e sem testes ativos, como gammimg e speedtest.

In [5]:
df_ow=df[df['scenario']=='OD']
df_ow_5g = df_ow[~df_ow['campaign'].str.endswith(('_gaming', '_speedtest'))]

In [6]:
df_ow_5g.shape

(604630, 37)

Verificação dos feixes nas redes das operadoras. Todas as operadoras estão usando multi-beams ou usam single-beam ?

In [7]:
unicos_mnc = df_ow_5g['MNC'].unique()
target_op2 = unicos_mnc[0]
target_op1 = unicos_mnc[1]
df_filtered_op2 = df_ow_5g[df_ow_5g['MNC'] == target_op2]

df_filtered_op1 = df_ow_5g[df_ow_5g['MNC'] == target_op1]
print('------------------------------------------------------------------------')
print(f'Operadora: {target_op1}')
ssb_counts_op1 = df_filtered_op1['SSBIdx'].value_counts().reset_index()
display(ssb_counts_op1)
ssb_counts_op2 = df_filtered_op2['SSBIdx'].value_counts().reset_index()
print('------------------------------------------------------------------------')
print(f'Operadora: {target_op2}')
display(ssb_counts_op2)

------------------------------------------------------------------------
Operadora: "Op"[1]


,SSBIdx,count
0,0,74386


------------------------------------------------------------------------
Operadora: "Op"[2]


,SSBIdx,count
0,2,74176
1,3,70235
2,1,69638
3,0,68062
4,5,64252
5,4,61864
6,7,61270
7,6,60747


Portanto, a rede da operadora ' "Op"[1] ' possui apenas 1 feixe e a rede da operadora ' "Op"[2] ' possui mais feixes, portanto apenas a segunda operadora tem utilidade para o ontext ode classificação de feixes.

In [8]:
df_ow_5g_op2=df_ow_5g[df_ow_5g['MNC']==target_op2]

In [9]:
df_ow_5g_op2.shape

(530244, 37)

In [18]:
df_ow_5g_op2.columns

Index(['Unnamed: 0', 'Date', 'Time', 'UTC', 'Latitude', 'Longitude',
       'Altitude', 'Speed', 'EARFCN', 'Frequency', 'Band', 'PCI', 'SSBIdx',
       'SSS-SINR', 'SSS-RSRP', 'SSS-RSRQ', 'SSS-RePower', 'MNC', 'DM_RS-SINR',
       'DM_RS-RSRP', 'DM_RS-RSRQ', 'DM_RS-RePower', 'PBCH-SINR', 'PBCH-RSRP',
       'PBCH-RSRQ', 'PBCH-RePower', 'PSS-SINR', 'PSS-RSRP', 'PSS-RSRQ',
       'PSS-RePower', 'SS_PBCH-SINR', 'SS_PBCH-RSRP', 'SS_PBCH-RSRQ',
       'SS_PBCH-RePower', 'scenario', 'distance_w', 'campaign', 'DateTime'],
      dtype='object')

Verificar se as features relacionadas a banda e frequencia são valores contínuos

In [19]:
df_ow_5g_op2[['EARFCN', 'Frequency', 'Band']].describe()

,EARFCN,Frequency,Band
count,530244.0,5.302440e+05,530244.0
mean,643296.0,3.649440e+03,78.0
std,0.0,4.073540e-08,0.0
min,643296.0,3.649440e+03,78.0
25%,643296.0,3.649440e+03,78.0
50%,643296.0,3.649440e+03,78.0
75%,643296.0,3.649440e+03,78.0
max,643296.0,3.649440e+03,78.0


In [20]:
df_ow_5g_op2=df_ow_5g_op2.drop(columns=['Unnamed: 0', 'Band', 'scenario','MNC','Frequency','EARFCN'])

In [21]:
df_ow_5g_op2.shape

(530244, 32)

In [24]:
print("\nLinhas com duplicadas:")
duplicate_rows = df_ow_5g_op2[df_ow_5g_op2.duplicated(keep=False)]
display(duplicate_rows)


Linhas com duplicadas:


,Date,Time,UTC,Latitude,Longitude,Altitude,Speed,PCI,SSBIdx,SSS-SINR,...,PSS-RSRP,PSS-RSRQ,PSS-RePower,SS_PBCH-SINR,SS_PBCH-RSRP,SS_PBCH-RSRQ,SS_PBCH-RePower,distance_w,campaign,DateTime


In [22]:
print("Linhas com valores ausentes:")
null_rows = df_ow_5g_op2[df_ow_5g_op2.isnull().any(axis=1)]
display(null_rows)

Rows with null values:


,Date,Time,UTC,Latitude,Longitude,Altitude,Speed,PCI,SSBIdx,SSS-SINR,...,PSS-RSRP,PSS-RSRQ,PSS-RePower,SS_PBCH-SINR,SS_PBCH-RSRP,SS_PBCH-RSRQ,SS_PBCH-RePower,distance_w,campaign,DateTime
5647184,17.12.2020,0 days 18:43:38.302000,1.610905e+09,41.86879,12.46499,16.34,37.87,86,1,-10.2,...,-128.7,-23.8,-119.0,-9.6,-128.7,-23.8,-119.0,NaN,campaign_50_OD_5G,2020-12-17 18:43:38.302
5721420,02.01.2021,0 days 12:32:35.811000,1.612266e+09,41.87088,12.46220,34.20,33.91,240,4,-8.1,...,-126.8,-17.7,-119.5,-7.9,-128.6,-19.3,-120.0,NaN,campaign_53_OD_5G,2021-01-02 12:32:35.811
5696457,11.01.2021,0 days 16:24:00.515000,1.613057e+09,41.86125,12.53370,45.52,44.17,473,2,-18.1,...,-127.3,-26.2,-111.9,NaN,NaN,NaN,NaN,25.118,campaign_51_OD_5G,2021-01-11 16:24:00.515
5696456,11.01.2021,0 days 16:24:00.515000,1.613057e+09,41.86125,12.53370,45.52,44.17,33,1,-18.3,...,-129.1,-27.6,-112.3,NaN,NaN,NaN,NaN,0.000,campaign_51_OD_5G,2021-01-11 16:24:00.515
5696498,11.01.2021,0 days 16:24:12.053000,1.613057e+09,41.86210,12.53313,44.15,5.33,33,0,-15.9,...,-126.7,-27.5,-110.0,NaN,NaN,NaN,NaN,0.000,campaign_51_OD_5G,2021-01-11 16:24:12.053
5701991,17.01.2021,0 days 17:51:39.874000,1.613581e+09,41.89855,12.42977,84.14,3.42,148,0,-21.6,...,-119.8,-33.1,-98.1,-20.7,-119.8,-32.6,-99.0,NaN,campaign_52_OD_5G,2021-01-17 17:51:39.874


In [25]:
print("Shape antes de excluir as linhas com valores nulos:", df_ow_5g_op2.shape)
df_ow_5g_op2.dropna(inplace=True)
print("Shape antes de excluir as linhas com valores nulos:", df_ow_5g_op2.shape)

Shape antes de excluir as linhas com valores nulos: (530244, 32)
Shape antes de excluir as linhas com valores nulos: (530238, 32)


# Analisando se o scaner capta apenas 1 sinal por instante ou se verifica muitplos possiveis sinais

Transformando a coluna de datas

In [13]:
df_ow_5g_op2['Time'] = pd.to_timedelta(df_ow_5g_op2['Time'])
df_ow_5g_op2['DateTime'] = pd.to_datetime(df_ow_5g_op2['Date'], format='%d.%m.%Y') + df_ow_5g_op2['Time']
df_ow_5g_op2 = df_ow_5g_op2.sort_values('DateTime')


/tmp/ipython-input-2486577727.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ow_5g_op2['Time'] = pd.to_timedelta(df_ow_5g_op2['Time'])
/tmp/ipython-input-2486577727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ow_5g_op2['DateTime'] = pd.to_datetime(df_ow_5g_op2['Date'], format='%d.%m.%Y') + df_ow_5g_op2['Time']


Verifica-se que para cada isntante de medição há mais de um sinal registrado. Isto ocorre pois o instrumento de emdição é um scanner de rede, ele não é um aparelho que inidica qual a conexao atual, apenas verifica quais as possibilidades em determinado ponto da medição.


In [14]:
min_datetime = df_ow_5g_op2['DateTime'].min()
df_ow_5g_op2[df_ow_5g_op2['DateTime'] == min_datetime]

,Unnamed: 0,Date,Time,UTC,Latitude,Longitude,Altitude,Speed,EARFCN,Frequency,...,PSS-RSRQ,PSS-RePower,SS_PBCH-SINR,SS_PBCH-RSRP,SS_PBCH-RSRQ,SS_PBCH-RePower,scenario,distance_w,campaign,DateTime
4747450,4310089,14.12.2020,0 days 18:49:24.612000,1.610647e+09,41.89322,12.49277,51.44,5.33,643296,3649.44,...,-18.4,-120.0,-7.7,-128.0,-19.1,-119.6,OD,0.0,campaign_34_OD_5G,2020-12-14 18:49:24.612
4747440,3310390,14.12.2020,0 days 18:49:24.612000,1.610647e+09,41.89322,12.49277,51.44,5.33,643296,3649.44,...,-14.3,-121.7,-1.7,-125.3,-14.9,-121.3,OD,0.0,campaign_34_OD_5G,2020-12-14 18:49:24.612
4747438,3110291,14.12.2020,0 days 18:49:24.612000,1.610647e+09,41.89322,12.49277,51.44,5.33,643296,3649.44,...,-20.8,-120.0,-6.8,-127.3,-18.3,-119.6,OD,0.0,campaign_34_OD_5G,2020-12-14 18:49:24.612
4747427,2010900,14.12.2020,0 days 18:49:24.612000,1.610647e+09,41.89322,12.49277,51.44,5.33,643296,3649.44,...,-24.2,-122.2,-16.8,-138.2,-27.6,-121.4,OD,0.0,campaign_34_OD_5G,2020-12-14 18:49:24.612


A filtragem dos sinais a cada instante de tempo será abordada no inicio do codigo "predicao_velocidade_v2.ipynb"

In [17]:
df_ow_5g_op2.shape

(530244, 38)

# Df final

In [ ]:
output_path = '/content/processed_df.parquet'
df.to_parquet(output_path)
print(f"DataFrame saved to {output_path}")